## Problem 3: How long distance individuals have travelled? (8 points)

In this problem the aim is to calculate the "distance" in meters that the individuals have travelled according the social media posts (Euclidean distances between points). In this problem, we will need the `userid` -column and the points created in the previous problem. You will need the shapefile `Kruger_posts.shp` generated in Problem 2 as input file.

Our goal is to answer these questions based on the input data:
- What was the shortest distance travelled in meters?
- What was the mean distance travelled in meters?
- What was the maximum distance travelled in meters?

**In your code, you should first:**
 - Import required modules.
 - Read in the shapefile as a geodataframe called `data`
 - Reproject the data from WGS84 projection into `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into the metric system.
 
*Store the result in a variable called `data`*!

In [2]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
from shapely.geometry import Point, LineString
import geopandas as gpd
import pandas as pd
from pyproj import CRS
fd = "Kruger_posts.shp"
data = gpd.read_file(fd)
data.to_crs(epsg=32735,inplace=True)



- Check the crs of the input data. If this information is missing, set it as epsg:4326 (WGS84).
- Reproject the data from WGS84 to `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system. (don't create a new variable, update the existing variable `data`!)"

In [3]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
if data.crs == None:
    data.to_crs(epsg=4326, inplace=True)

data.to_crs(epsg=32735, inplace=True)

In [4]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                         geometry  
0  POINT (952912.890 7229683.258)  
1  POINT (953433.223 7172080.632)  
2  POINT (898955.144 7302197.408)  
3  POINT (956927.218 7243564.942)  
4  POINT (956794.955 7236187.926)  


In [5]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
print(data.crs)

epsg:32735


 - Group the data by userid

In [6]:
grouped = data.groupby("userid")

In [7]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the number of groups:
assert len(grouped.groups) == data["userid"].nunique(), "Number of groups should match number of unique users!"

**Create LineString objects for each user connecting the points from oldest to latest:**

*Suggested steps:*
- Create an empty DataFrame called `movements`. 
- Create an empty column "geometry"
- Use a for-loop where you iterate over the grouped object. For each user's data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a LineString object based on the user's points
    - Add the LineString to the geometry column of the `movements` dataframe. You can also add the `userid` in a separate column (or use the userid as index).
- Convert `movements` into a `GeoDataFrame` (you can replace the DataFrame created in the previous steps with the GeoDataFrame). Remember to set the `geometry` column.
- Set the CRS of the ``movements`` GeoDataFrame as ``EPSG:32735`` 

In [8]:
movements = pd.DataFrame(columns=["geometry"])
for user, rows in grouped:
    rows.sort_values(by="timestamp",inplace=True)
    
    # HT comment: This works, but here you could have just used the `rows` variable that contains the group of values for a specific user
    group = grouped.get_group(user)
    if len(group["geometry"]) >=2:
        line = LineString(list(group["geometry"]))
        movements.at[user,"userid"] = user
        movements.at[user,"geometry"] = line
movements = gpd.GeoDataFrame(movements, crs=32735, geometry=movements["geometry"])
    
    

    

In [9]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the result
print(type(movements))
print(movements.crs)
print(movements["geometry"].head())

<class 'geopandas.geodataframe.GeoDataFrame'>
epsg:32735
16301    LINESTRING (939011.113 7254636.121, 942231.630...
45136    LINESTRING (905394.500 7193375.148, 905394.500...
50136    LINESTRING (963788.403 7228015.063, 944551.607...
88775    LINESTRING (902800.817 7192546.975, 902800.839...
88918    LINESTRING (959332.961 7219877.715, 963788.403...
Name: geometry, dtype: geometry


**Finally:**
- Check once more the crs definition of your dataframe (should be epsg:32735, define the correct crs if this information is missing)
- Calculate the lenghts of the lines into a new column called ``distance`` in ``movements`` GeoDataFrame.

In [11]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
if movements.crs != 32735:
    data.to_crs(epsg=32735, inplace=True)
movements["distance"] = movements["geometry"].length


In [12]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

,geometry,userid,distance
16301,"LINESTRING (939011.113 7254636.121, 942231.630...",16301.0,195251.395657
45136,"LINESTRING (905394.500 7193375.148, 905394.500...",45136.0,0.000000
50136,"LINESTRING (963788.403 7228015.063, 944551.607...",50136.0,254702.529630
88775,"LINESTRING (902800.817 7192546.975, 902800.839...",88775.0,0.080245
88918,"LINESTRING (959332.961 7219877.715, 963788.403...",88918.0,9277.252211


You should now be able to print answers to the following questions: 

 - What was the shortest distance travelled in meters?
 - What was the mean distance travelled in meters?
 - What was the maximum distance travelled in meters?

In [16]:
distance = movements["distance"]

print(min(distance))
#0.0
print(distance.mean())
#69090.38226302531
print(max(distance))
#69090.38226302531

0.0
69090.38226302531
4535318.9896208625


- Finally, save the movements of into a Shapefile called ``some_movements.shp``

In [17]:
fp = 'some_movements.shp'
movements.to_file(fp)

In [18]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

import os

#Check if output file exists
assert os.path.isfile(fp), "Output file does not exits."

That's all for this week!